# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [87]:
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import sum as Fsum

In [25]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, functions

# TODOS: 
# 1) import any other libraries you might need
# 4) write code to answer the quiz questions 

# 2) instantiate a Spark session 
spark = (
    SparkSession
    .builder
    .appName("data_wrangling_quiz")
    .getOrCreate()
)

# 3) read in the data set located at the path "data/sparkify_log_small.json"
data = spark.read.load("data/sparkify_log_small.json", format="json")

In [26]:
data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [27]:
data.createOrReplaceTempView("userLog")

# Question 1

Which page did user id "" (empty string) NOT visit?

In [10]:
# TODO: write your code to answer question 1
data.filter(data["userId"] == "").groupBy("page").count().show()

+-----+-----+
| page|count|
+-----+-----+
| Home|  187|
|About|   15|
|Login|  126|
| Help|    8|
+-----+-----+



In [78]:
data.filter(data["userId"] != "").groupBy("page").count().show()

+----------------+-----+
|            page|count|
+----------------+-----+
|Submit Downgrade|    1|
|            Home|  939|
|       Downgrade|   75|
|          Logout|  100|
|   Save Settings|   11|
|           About|   28|
|        Settings|   59|
|        NextSong| 8347|
|            Help|   50|
|         Upgrade|   32|
|           Error|   12|
|  Submit Upgrade|   10|
+----------------+-----+



In [16]:
spark.sql(
    """
    SELECT
        page,
        COUNT(*) AS num_visit
    FROM
        userLog
    WHERE
        userId = ''
    GROUP BY 1
    """
).show()

+-----+---------+
| page|num_visit|
+-----+---------+
| Home|      187|
|About|       15|
|Login|      126|
| Help|        8|
+-----+---------+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [2]:
# TODO: use this space to explore the behavior of the user with an empty string
# unregistered visitors

# Question 3

How many female users do we have in the data set?

In [31]:
%%time
data.groupBy("gender").agg(countDistinct("userId")).show()

+------+----------------------+
|gender|count(DISTINCT userId)|
+------+----------------------+
|     F|                   462|
|  null|                     1|
|     M|                   501|
+------+----------------------+

CPU times: user 6.01 ms, sys: 0 ns, total: 6.01 ms
Wall time: 4.68 s


In [33]:
%%time
# TODO: write your code to answer question 3
spark.sql("""
    SELECT
        gender,
        COUNT(DISTINCT userId) as num_users
    FROM
        userLog
    GROUP BY 1
""").show()

+------+---------+
|gender|num_users|
+------+---------+
|     F|      462|
|  null|        1|
|     M|      501|
+------+---------+

CPU times: user 2.48 ms, sys: 0 ns, total: 2.48 ms
Wall time: 4.12 s


# Question 4

How many songs were played from the most played artist?

In [34]:
data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [47]:
(
    data
    .groupBy("artist")
    .agg(countDistinct("sessionId").alias("count"))
    .filter(~data["artist"].isNull())
    .sort(desc("count"))
    .show()
)

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|            Coldplay|   68|
|       Kings Of Leon|   57|
|Florence + The Ma...|   51|
|            BjÃÂ¶rk|   42|
|       Dwight Yoakam|   40|
|      The Black Keys|   39|
|       Justin Bieber|   37|
|         OneRepublic|   34|
|                Muse|   33|
|        Jack Johnson|   32|
|        Taylor Swift|   28|
|           Radiohead|   28|
|          Nickelback|   27|
|Barry Tuckwell/Ac...|   27|
|           Metallica|   26|
|               Train|   25|
|          Lily Allen|   25|
|     Alliance Ethnik|   24|
|           Daft Punk|   24|
|Red Hot Chili Pep...|   23|
+--------------------+-----+
only showing top 20 rows



In [45]:
data.filter(data["artist"] == "Coldplay").agg(count("song")).show()

+-----------+
|count(song)|
+-----------+
|         83|
+-----------+



In [53]:
spark.sql("""
        SELECT
            artist
        FROM 
            (SELECT
                artist,
                COUNT(DISTINCT sessionId) AS num_sessions
            FROM
                userLog
            WHERE
                artist <> 'null'
            GROUP BY 1
            ORDER BY 2 DESC)
        LIMIT 1
""").show()

+--------+
|  artist|
+--------+
|Coldplay|
+--------+



In [80]:
(
    data
    .filter(data.page == "NextSong")
    .select("artist")
    .groupBy("artist")
    .agg(count("artist").alias("artistCount"))
    .sort(desc("artistCount"))
    .show(5)
)

+--------------------+-----------+
|              artist|artistCount|
+--------------------+-----------+
|            Coldplay|         83|
|       Kings Of Leon|         69|
|Florence + The Ma...|         52|
|            BjÃÂ¶rk|         46|
|       Dwight Yoakam|         45|
+--------------------+-----------+
only showing top 5 rows



In [ ]:
# TODO: write your code to answer question 4


# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [56]:
data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [82]:
function = udf(lambda ishome: int(ishome == "Home"), IntegerType())

In [84]:
user_window = (
    Window
    .partitionBy("userId")
    .orderBy(desc("ts"))
    .rangeBetween(Window.unboundedPreceding, 0)
)

In [88]:
cusum = (
    data
    .filter((df.page == "NextSong") | (df.page == "Home"))
    .select("userId", "page", "ts")
    .withColumn("homeVisit", function("page"))
    .withColumn("period", Fsum("homeVisit").over(user_window))
)

In [93]:
cusum.filter(cusum.userId == "1046").show(50)

+------+--------+-------------+---------+------+
|userId|    page|           ts|homeVisit|period|
+------+--------+-------------+---------+------+
|  1046|NextSong|1513842410284|        0|     0|
|  1046|NextSong|1513842186284|        0|     0|
|  1046|NextSong|1513841947284|        0|     0|
|  1046|NextSong|1513841710284|        0|     0|
|  1046|NextSong|1513841519284|        0|     0|
|  1046|NextSong|1513841332284|        0|     0|
|  1046|NextSong|1513841100284|        0|     0|
|  1046|NextSong|1513840929284|        0|     0|
|  1046|NextSong|1513840740284|        0|     0|
|  1046|NextSong|1513840267284|        0|     0|
|  1046|NextSong|1513840086284|        0|     0|
|  1046|NextSong|1513839785284|        0|     0|
|  1046|NextSong|1513839573284|        0|     0|
|  1046|NextSong|1513839321284|        0|     0|
|  1046|NextSong|1513839014284|        0|     0|
|  1046|NextSong|1513838520284|        0|     0|
|  1046|NextSong|1513838300284|        0|     0|
|  1046|    Home|151

In [99]:
(
    cusum
    .filter((cusum.page == "NextSong") & (cusum.userId == "1046"))
    .groupBy("userId", "period")
    .agg({"period": "count"})
#     .agg({"count(period)": "avg"})
).show()

+------+------+-------------+
|userId|period|count(period)|
+------+------+-------------+
|  1046|     0|           17|
|  1046|     1|            3|
|  1046|     2|            7|
+------+------+-------------+



In [98]:
(
    cusum
    .filter(cusum.page == "NextSong")
    .groupBy("userId", "period")
    .agg({"period": "count"})
    .agg({"count(period)": "avg"})
).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+



In [70]:
df = data.filter(data["userId"] == 1046)
df.select(df.userId, df.page, df.song, df.sessionId).show(50)

+------+--------+--------------------+---------+
|userId|    page|                song|sessionId|
+------+--------+--------------------+---------+
|  1046|NextSong|Christmas Tears W...|     5132|
|  1046|NextSong|  Be Wary Of A Woman|     5132|
|  1046|NextSong|   Public Enemy No.1|     5132|
|  1046|NextSong|Reign Of The Tyrants|     5132|
|  1046|NextSong|      Father And Son|     5132|
|  1046|NextSong|               No. 5|     5132|
|  1046|NextSong|           Seventeen|     5132|
|  1046|    Home|                null|     5786|
|  1046|NextSong|          War on war|     5786|
|  1046|NextSong|   Killermont Street|     5786|
|  1046|NextSong|        Black & Blue|     5786|
|  1046|  Logout|                null|     5786|
|  1046|    Home|                null|     5786|
|  1046|NextSong|     Heads Will Roll|     5786|
|  1046|NextSong|Bleed It Out [Liv...|     5786|
|  1046|NextSong|              Clocks|     5786|
|  1046|NextSong|           Love Rain|     5786|
|  1046|NextSong|Ry 

In [76]:
df = data.filter(data["userId"] == 597)
df.select(df.userId, df.page, df.song, df.sessionId).show(50)

+------+--------+--------------------+---------+
|userId|    page|                song|sessionId|
+------+--------+--------------------+---------+
|   597|    Home|                null|     3689|
|   597|NextSong|Sheebeg And Sheemore|     3689|
|   597|NextSong|Quelqu'un M'a Dit...|     3689|
+------+--------+--------------------+---------+



In [ ]:
# TODO: write your code to answer question 5